<a href="https://colab.research.google.com/github/AnaJuliaMM/final_chatbot/blob/develop/taskbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Dependências Externas

In [7]:
!pip install --upgrade nltk
!pip install fuzzywuzzy

In [6]:
import pandas as pd
import string
import nltk
from dataclasses import dataclass, field
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from typing import List, Optional
from fuzzywuzzy import fuzz

# Baixar os pacotes necessários do NLTK
nltk.download('punkt_tab')
nltk.download('rslp')
nltk.download('stopwords')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Dados para consulta

In [8]:

# Dados iniciais
tasks = {
    'task_id': [1, 2, 3, 4, 5],
    'task_name': [
        'Comprar leite',
        'Enviar e-mail',
        'Reunião com a equipe',
        'Revisar relatório',
        'Organizar arquivos'
    ],
    'status': ['pendente', 'concluída', 'pendente', 'pendente', 'concluída'],
    'created_at': pd.to_datetime([
        '2023-10-01 09:00:00',
        '2023-10-01 10:00:00',
        '2023-10-02 11:00:00',
        '2023-10-03 12:30:00',
        '2023-10-04 14:15:00'
    ]),
    'due_date': pd.to_datetime([
        '2023-10-05',
        '2023-10-02',
        '2023-10-04',
        '2023-10-05',
        '2023-10-06'
    ])
}

tasks_df = pd.DataFrame(tasks)

## Estrutura da Árvore de Decisão

### Classe Node

In [ ]:
@dataclass
class Node:
    """Representa um nó da árvore com um valor e filhos."""
    value: str
    children: List['Node'] = field(default_factory=list)

### Classe Tree


In [ ]:
class Tree:
    def __init__(self, value: str, stemmer=None):
        self.root = Node(value)
        self.stemmer = stemmer

    def _stem_if_needed(self, value: str) -> str:
        """Aplica o stemming ao valor se o stemmer estiver configurado."""
        return self.stemmer.stem(value) if self.stemmer else value

    def add_child(self, parent_value: str, value: str):
        parent_value = self._stem_if_needed(parent_value)
        value = self._stem_if_needed(value)
        parent_node = self.search(parent_value)
        if not parent_node:
            raise ValueError(f"Nó pai '{parent_value}' não encontrado.")
        if not any(child.value == value for child in parent_node.children):
            parent_node.children.append(Node(value))
            print(f"Adicionado nó: {value} como filho de: {parent_value}")

    def search(self, value: str, node: Optional[Node] = None) -> Optional[Node]:
        if node is None:
            node = self.root
        if self._match_value(node.value, value):
            return node
        for child in node.children:
            result = self.search(value, child)
            if result:
                return result
        return None

    def _match_value(self, node_value: str, search_value: str) -> bool:
        search_value = self._stem_if_needed(search_value)
        similarity = fuzz.ratio(node_value, search_value)
        return similarity > 80

### Instãncia da árvore de decisão

In [9]:

# Instanciar árvore de decisão
stemmer = RSLPStemmer()
tree = Tree("chatbot", stemmer)

# Preencher a árvore de decisão
tree.add_child("chatbot", "taref")  # Stemming de "tarefa"
tree.add_child("taref", "adicion")  # Stemming de "adicionar"
tree.add_child("taref", "list")     # Stemming de "listar"
tree.add_child("taref", "encontr")  # Stemming de "encontrar"
tree.add_child("taref", "conclu")   # Stemming de "concluir"
tree.add_child("taref", "exclu")    # Stemming de "excluir"

# Adicionar a entidade "status" e sua intenção "alterar"
tree.add_child("chatbot", "status")  # Stemming de "status"
tree.add_child("status", "alter")   # Stemming de "alterar"

Adicionado nó: taref como filho de: chatbot
Adicionado nó: adicion como filho de: taref
Adicionado nó: list como filho de: taref
Adicionado nó: encontr como filho de: taref
Adicionado nó: conclu como filho de: taref
Adicionado nó: exclu como filho de: taref
Adicionado nó: statu como filho de: chatbot
Adicionado nó: alt como filho de: statu


### Funções de manipulação dos dados

In [10]:
# Funções associadas
def add_task_to_df():
    """
    Adiciona uma nova tarefa ao DataFrame global tasks_df.
    """
    global tasks_df
    try:
        task_name = input("Digite o nome da tarefa: ").strip()
        due_date = input("Digite a data final da tarefa (YYYY-MM-DD): ").strip()
        due_date = pd.to_datetime(due_date, format='%Y-%m-%d', errors='coerce')
        if pd.isna(due_date):
            print("Data inválida. Por favor, use o formato YYYY-MM-DD.")
            return
        new_task = {
            'task_id': tasks_df['task_id'].max() + 1 if not tasks_df.empty else 1,
            'task_name': task_name,
            'status': 'pendente',
            'created_at': pd.Timestamp.now(),
            'due_date': due_date
        }
        tasks_df = pd.concat([tasks_df, pd.DataFrame([new_task])], ignore_index=True)
        print(f"Tarefa '{task_name}' adicionada com sucesso!")
        print("Agora as suas tarefas estão assim:")
        list_tasks()
    except Exception as e:
        print(f"Erro ao adicionar tarefa: {e}")

In [11]:
def list_tasks():
    """
    Lista todas as tarefas do DataFrame global tasks_df.
    """
    print("Tarefas no sistema:")
    print(tasks_df)

In [12]:
def delete_task():
    """
    Deleta uma tarefa do DataFrame global tasks_df com base no ID fornecido pelo usuário.
    Antes de deletar, lista todas as tarefas disponíveis.
    """
    global tasks_df
    try:
        # Listar todas as tarefas antes de solicitar o ID
        print("Aqui estão todas as tarefas disponíveis:")
        list_tasks()

        task_id = int(input("Digite o ID da tarefa a ser deletada: ").strip())
        if task_id in tasks_df['task_id'].values:
            tasks_df = tasks_df[tasks_df['task_id'] != task_id].reset_index(drop=True)
            print(f"Tarefa com ID {task_id} deletada com sucesso!")
            print("Agora as suas tarefas estão assim:")
            list_tasks()
        else:
            print(f"Nenhuma tarefa encontrada com o ID {task_id}.")
    except ValueError:
        print("ID inválido. Por favor, insira um número inteiro.")
    except Exception as e:
        print(f"Erro ao deletar tarefa: {e}")


In [13]:
def alter_task_status():
    """
    Altera o status de uma tarefa no DataFrame global tasks_df com base no ID fornecido pelo usuário.
    Antes de alterar, lista todas as tarefas disponíveis.
    """
    global tasks_df
    try:
        # Listar todas as tarefas antes de solicitar o ID
        print("Aqui estão todas as tarefas disponíveis:")
        list_tasks()

        task_id = int(input("Digite o ID da tarefa que deseja alterar: ").strip())
        if task_id in tasks_df['task_id'].values:
            print("Escolha o novo status para a tarefa:")
            print("P - Pendente")
            print("C - Concluída")
            status_option = input("Digite o número correspondente ao novo status: ").strip()

            if status_option == 'P':
                new_status = 'pendente'
            elif status_option == 'C':
                new_status = 'concluída'
            else:
                print("Opção inválida. O status não foi alterado.")
                return

            tasks_df.loc[tasks_df['task_id'] == task_id, 'status'] = new_status
            print(f"O status da tarefa com ID {task_id} foi alterado para '{new_status}'.")
            print("Agora as suas tarefas estão assim:")
            list_tasks()
        else:
            print(f"Nenhuma tarefa encontrada com o ID {task_id}.")
            list_tasks()
    except ValueError:
        print("ID inválido. Por favor, insira um número inteiro.")
    except Exception as e:
        print(f"Erro ao alterar o status da tarefa: {e}")

### Mapeamento de intenções para funções

In [14]:
intent_function_map = {
    'adicion': add_task_to_df,
    'list': list_tasks,
    'exclu': delete_task,
    'alter': alter_task_status
}

## Funções de processamento do prompt do usuário

In [15]:
def preprocess_prompt(prompt: str) -> List[str]:
    tokens = word_tokenize(prompt.lower())
    tokens = [word for word in tokens if word not in stopwords.words('portuguese') and word not in string.punctuation]
    tokens = [stemmer.stem(token) for token in tokens]
    return tokens

In [16]:
def process_prompt(prompt: str):
    """
    Processa o prompt do usuário, identificando entidade e intenção,
    e executa a função correspondente.
    """
    tokens = preprocess_prompt(prompt)
    print(f"Tokens processados: {tokens}")
    entity, intent = None, None

    for token in tokens:
        if not entity and tree.search(token):
            entity = token
            break

    if entity:
        for token in tokens:
            if tree.search(token, tree.search(entity)):
                intent = token
                break

    if entity and intent:
        action = intent_function_map.get(intent)
        if action:
            action()
        else:
            print("Ação não encontrada para a intenção.")
    else:
        print("Desculpe, não consegui entender o que você deseja.")

### Função principal

In [17]:
def main():
    print("Olá, seja bem-vindo(a) ao chatbot de tarefas!")
    print("Atualmente você tem essas tarefas:")
    list_tasks()
    while True:
        prompt = input("O que você gostaria de fazer? (Adicionar tarefa | Alterar Status | Listar tarefas | Excluir tarefas | Sair para encerrar o chat): ")
        if prompt.lower() == 'sair':
            print("Até mais!")
            break
        process_prompt(prompt)

In [18]:
main()

Olá, seja bem-vindo(a) ao chatbot de tarefas!
Atualmente você tem essas tarefas:
Tarefas no sistema:
   task_id             task_name     status          created_at   due_date
0        1         Comprar leite   pendente 2023-10-01 09:00:00 2023-10-05
1        2         Enviar e-mail  concluída 2023-10-01 10:00:00 2023-10-02
2        3  Reunião com a equipe   pendente 2023-10-02 11:00:00 2023-10-04
3        4     Revisar relatório   pendente 2023-10-03 12:30:00 2023-10-05
4        5    Organizar arquivos  concluída 2023-10-04 14:15:00 2023-10-06
O que você gostaria de fazer? (Adicionar tarefa | Alterar Status | Listar tarefas | Excluir tarefas | Sair para encerrar o chat): adicionar tarefa
Tokens processados: ['adicion', 'taref']
Digite o nome da tarefa: aaaaaaaaaaa
Digite a data final da tarefa (YYYY-MM-DD): 2024-10-12
Tarefa 'aaaaaaaaaaa' adicionada com sucesso!
Agora as suas tarefas estão assim:
Tarefas no sistema:
   task_id             task_name     status                 created_a